In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
import numpy as np

In [33]:
# Ratings DataFrame + cleaning

df_brands = pd.read_csv("ratings.csv")
df_brands["Overall rating"] = df_brands["Overall rating"].apply(lambda x: x.replace("Rated: ", ""))

# Add information about the numerical overall rating
rating_mapping = {"We avoid": 1,
                  "Not good enough": 2,
                  "It's a start": 3,
                  "Good": 4,
                  "Great": 5
                 }
df_brands["Overall rating numerical"] = df_brands["Overall rating"].map(rating_mapping)

# Create a new column with index positions at the Lyst Index 
lyst_rank = {"Miu Miu": 1,
            "Loewe": 2,
            "Saint Laurent": 3,
            "Prada": 4,
            "Coach": 5,
            "The Row": 6,
            "Cos": 7,
            "Chloe": 8,
            "Bottega Veneta": 9,
            "Moncler": 10,
            "Ralph Lauren": 11,
            "Alaia": 12,
            "Balenciaga": 13,
            "Jacquemus": 14,
            "Skims": 15,
            "Versace": 16,
            "Burberry": 17,
            "Gucci": 18,
            "Birkenstock": 19,
            "Valentino": 20
            }
df_brands["Lyst position"] = df_brands["Brand"].map(lyst_rank)

# Create a new column with holding company names
holding_company = {"Miu Miu": "Prada Group",
                     "Loewe": "LVMH",
                     "Saint Laurent": "Kering",
                     "Prada": "Prada Group",
                     "Coach": "Tapestry Inc.",
                     "The Row": "Private - Olsen twins",
                     "Cos": "H&M Group",
                     "Chloe": "Richemont",
                     "Bottega Veneta": "Kering",
                     "Moncler": "Moncler S.p.A.",
                     "Ralph Lauren": "Ralph Lauren Corporation",
                     "Alaia": "Richemont",
                     "Balenciaga": "Kering",
                     "Jacquemus": "Private - Simon Porte Jacquemus",
                     "Skims": "Private - Kim Kardashian",
                     "Versace": "Capri Holdings",
                     "Burberry": "Burberry Group plc",
                     "Gucci": "Kering",
                     "Birkenstock": "Private Equity - L Catterton & Financière Agache",  
                     "Valentino": "Private Equity - Mayhoola"
                     }      
df_brands["Holding"] = df_brands["Brand"].map(holding_company)

# Create a new column with information about market exchanges
exchanges = {"Miu Miu": "Hong Kong Stock Exchange",
                     "Loewe": "Euronext Paris",
                     "Saint Laurent": "Euronext Paris",
                     "Prada": "Hong Kong Stock Exchange",
                     "Coach": "NYSE",
                     "The Row": "",
                     "Cos": "Nasdaq Stockholm",
                     "Chloe": "SIX Swiss Exchange",
                     "Bottega Veneta": "Euronext Paris",
                     "Moncler": "Borsa Italiana",
                     "Ralph Lauren": "NYSE",
                     "Alaia": "SIX Swiss Exchange",
                     "Balenciaga": "Euronext Paris",
                     "Jacquemus": "",
                     "Skims": "",
                     "Versace": "NYSE",
                     "Burberry": "London Stock Exchange",
                     "Gucci": "Euronext Paris",
                     "Birkenstock": "NYSE",  
                     "Valentino": ""
                     }      
df_brands["Market exchange"] = df_brands["Brand"].map(exchanges)

# Create a new column with tickers of holding companies
tickers_holding_company = {"Miu Miu": "1913.HK",
                     "Loewe": "MC.PA",
                     "Saint Laurent": "KER.PA",
                     "Prada": "1913.HK",
                     "Coach": "TPR",
                     "The Row": "",
                     "Cos": "HM-B.ST",
                     "Chloe": "CFR.SW",
                     "Bottega Veneta": "KER.PA",
                     "Moncler": "MONC.MI",
                     "Ralph Lauren": "RL",
                     "Alaia": "CFR.SW",
                     "Balenciaga": "KER.PA",
                     "Jacquemus": "",
                     "Skims": "",
                     "Versace": "CPRI",
                     "Burberry": "BRBY.L",
                     "Gucci": "KER.PA",
                     "Birkenstock": "BIRK",  
                     "Valentino": ""
                     }      
df_brands["Ticker holding"] = df_brands["Brand"].map(tickers_holding_company)

# Reorder columns
df_brands = df_brands[['Lyst position','Brand', 'Holding','Market exchange', 'Ticker holding','Overall rating numerical', 'Overall rating', 'Planet rating', 'People rating',
       'Animals rating']]

df_brands


,Lyst position,Brand,Holding,Market exchange,Ticker holding,Overall rating numerical,Overall rating,Planet rating,People rating,Animals rating
0,1,Miu Miu,Prada Group,Hong Kong Stock Exchange,1913.HK,2,Not good enough,3 out of 5,3 out of 5,1 out of 5
1,2,Loewe,LVMH,Euronext Paris,MC.PA,2,Not good enough,3 out of 5,2 out of 5,1 out of 5
2,3,Saint Laurent,Kering,Euronext Paris,KER.PA,3,It's a start,4 out of 5,3 out of 5,2 out of 5
3,4,Prada,Prada Group,Hong Kong Stock Exchange,1913.HK,3,It's a start,3 out of 5,3 out of 5,2 out of 5
4,5,Coach,Tapestry Inc.,NYSE,TPR,2,Not good enough,3 out of 5,2 out of 5,1 out of 5
5,6,The Row,Private - Olsen twins,,,1,We avoid,1 out of 5,1 out of 5,2 out of 5
6,7,Cos,H&M Group,Nasdaq Stockholm,HM-B.ST,3,It's a start,3 out of 5,2 out of 5,3 out of 5
7,8,Chloe,Richemont,SIX Swiss Exchange,CFR.SW,3,It's a start,3 out of 5,3 out of 5,2 out of 5
8,9,Bottega Veneta,Kering,Euronext Paris,KER.PA,3,It's a start,4 out of 5,4 out of 5,2 out of 5
9,10,Moncler,Moncler S.p.A.,Borsa Italiana,MONC.MI,3,It's a start,4 out of 5,2 out of 5,2 out of 5


In [34]:
grouped = df_brands.groupby(['Holding', 'Ticker holding'])['Brand'].apply(', '.join).reset_index()
grouped

,Holding,Ticker holding,Brand
0,Burberry Group plc,BRBY.L,Burberry
1,Capri Holdings,CPRI,Versace
2,H&M Group,HM-B.ST,Cos
3,Kering,KER.PA,"Saint Laurent, Bottega Veneta, Balenciaga, Gucci"
4,LVMH,MC.PA,Loewe
5,Moncler S.p.A.,MONC.MI,Moncler
6,Prada Group,1913.HK,"Miu Miu, Prada"
7,Private - Kim Kardashian,,Skims
8,Private - Olsen twins,,The Row
9,Private - Simon Porte Jacquemus,,Jacquemus


In [35]:
# Extract tickers
tickers = grouped['Ticker holding'].unique()

# Prepare storage for data
all_data = []

# Loop through tickers and fetch data
for ticker in tickers:
    try:
        # Fetch historical data
        ticker_data = yf.Ticker(ticker).history(period='5y')
        ticker_data['Ticker'] = ticker  # Add a column with the ticker
        
        # Currency information
        info = yf.Ticker(ticker).info
        ticker_data['Currency'] = info.get('currency', 'N/A')  # Add currency information
        
        # Append the data to the list
        all_data.append(ticker_data)
        print(f"Data for {ticker} retrieved successfully.")

    except Exception as e:
        print(f"Failed to retrieve data for {ticker}: {e}")

# Concatenate all data into a single DataFrame
if all_data:
    big_data = pd.concat(all_data)
    big_data.reset_index(inplace=True)  # Reset index to have a flat structure
else:
    big_data = pd.DataFrame()  # Return empty DataFrame if no data

Data for BRBY.L retrieved successfully.
Data for CPRI retrieved successfully.
Data for HM-B.ST retrieved successfully.
Data for KER.PA retrieved successfully.
Data for MC.PA retrieved successfully.
Data for MONC.MI retrieved successfully.
Data for 1913.HK retrieved successfully.
Failed to retrieve data for : Empty ticker name
Data for BIRK retrieved successfully.
Data for RL retrieved successfully.
Data for CFR.SW retrieved successfully.
Data for TPR retrieved successfully.


In [ ]:
big_data_agg = big_data.groupby(['Ticker', 'Currency']).agg({'Close': ['mean', 'min', 'max'], 'Volume': 'sum'})

# Let's format the values 
big_data_agg = big_data_agg.round(2)
big_data_agg.columns = ['_'.join(col).strip() for col in big_data_agg.columns.values]
big_data_agg.reset_index(inplace=True)  

big_data_agg

,Ticker,Currency,Close_mean,Close_min,Close_max,Volume_sum
0,1913.HK,HKD,47.67,28.03,68.35,1581957051
1,BIRK,USD,49.74,36.38,63.57,743656258
2,BRBY.L,GBp,1600.03,571.00,2638.95,2193658490
3,CFR.SW,CHF,115.92,50.63,181.19,1469715262
4,CPRI,USD,42.15,12.48,70.76,3112053791
5,HM-B.ST,SEK,141.31,87.06,185.98,4267000080
6,KER.PA,EUR,420.69,153.52,703.42,311055078
7,MC.PA,EUR,631.63,370.23,858.37,542647757
8,MONC.MI,EUR,51.82,31.48,67.55,998540128
9,RL,USD,145.70,60.57,334.98,1168527187
